## This notebook shows how to fit a four image lens with (or without) substructure in the model

In [2]:
from lenstronomywrapper.LensSystem.quad_lens import QuadLensSystem
from lenstronomywrapper.LensSystem.BackgroundSource.quasar import Quasar

from lenstronomywrapper.LensSystem.LensComponents.powerlawshear import PowerLawShear
from lenstronomywrapper.LensSystem.macrolensmodel import MacroLensModel
from lenstronomywrapper.LensData.lensed_quasar import LensedQuasar

from lenstronomywrapper.Optimization.quad_optimization.hierarchical import HierarchicalOptimization
from lenstronomywrapper.Optimization.quad_optimization.brute import BruteOptimization

from pyHalo.pyhalo import pyHalo
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

### First we create a realization of dark matter halos using pyHalo (see the example notebook in the pyHalo package)

In [3]:
zlens, zsource = 0.5, 1.5
cone_opening_angle = 6.
rendering_volume = 'DOUBLE_CONE' 
kwargs_halo_mass_function = {'geometry_type': rendering_volume}
mass_definition = 'TNFW' 
log_mlow = 7. 
log_mhigh = 10 
power_law_index = -1.9 
LOS_norm = 1. 
sigma_sub = 0.02 
r_tidal = '0.25Rs' 
log_m_host = 13.

log_mass_sheet_min = 6
log_mass_sheet_max = 10

pyhalo = pyHalo(zlens, zsource, kwargs_halo_mass_function=kwargs_halo_mass_function)

realization_kwargs = {'mass_func_type': 'POWER_LAW', 'log_mlow': log_mlow, 'log_mhigh': log_mhigh, 
                      'log_mass_sheet_min': log_mlow, 'log_mass_sheet_max': log_mhigh, 
                      'mdef_main': mass_definition,'mdef_los': mass_definition, 'sigma_sub': sigma_sub,
                      'cone_opening_angle': cone_opening_angle, 
                      'log_m_host': log_m_host, 'power_law_index': power_law_index, 'r_tidal': r_tidal,
                      'LOS_normalization': LOS_norm}

realization_type = 'composite_powerlaw'
astropy_instance = pyhalo.astropy_cosmo
realization_init = pyhalo.render(realization_type, realization_kwargs, nrealizations=1)[0]
lens_model_list, lens_redshift_array, kwargs_halos, _ = realization_init.lensing_quantities()

### Now define a macromodel

In [10]:
zlens, zsource = 0.5, 1.5
kwargs_macromodel = [{'theta_E': 1., 'center_x': 0., 'center_y': -0.0, 'e1': 0.1, 'e2': 0.2, 'gamma': 2.0}, 
               {'gamma1': 0.04, 'gamma2': 0.07}]

main_lens_fit = PowerLawShear(zlens, kwargs_macromodel)
macromodel = MacroLensModel([main_lens_fit])

source_size_parsec = 30. # FWHM of a Gaussian
kwargs_source = {'center_x': 0., 'center_y': 0., 'source_fwhm_pc': source_size_parsec}
background_source = Quasar(kwargs_source)

### Now we can create a lens system

In [18]:
lens_system = QuadLensSystem(macromodel, zsource, background_source, realization_init)
# Can save it for later if you like
# import dill
# f = open('example_lens_system', 'wb')
# dill.dump(lens_system, f)
# f.close()

### You can optionally use the class method shift_background_auto to align the halos with the path traversed by lensed light rays
#### For more info on this, see the example notebook in pyHalo/lensed_images_with_shifted_background_halos

In [16]:
# this is the data for HE0435
x_image = np.array([ 1.272,  0.306, -1.152, -0.384]) 
y_image = np.array([ 0.156, -1.092, -0.636,  1.026])
flux_ratios = np.array([[0.96,  0.976, 1., 0.65 ]])

lens_data = LensedQuasar(x_image, y_image, flux_ratios)
lens_system = QuadLensSystem.shift_background_auto(lens_data, macromodel, zsource, background_source, realization_init)

lensmodel, kwargs_lens = lens_system.get_lensmodel()
print(lensmodel.lens_model_list)

['EPL', 'SHEAR', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW',